In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir("/content/gdrive/My Drive/wzk/seg1")
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install ttach

## 引入相关库

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
from torchsummary import summary
import torch.nn as nn
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as tf
from losses import *
from utils import *
from UNet import *
from UNetplusplus import *
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
# import segmentation_models_pytorch as smp
import ttach as tta
import random

In [4]:
# 设置随机数种子
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
seed = 1024
setup_seed(seed)

## 读取数据

In [5]:
# 读取数据
train_x_path = './supplementary_modify/dataset1/train/'
train_y_path = './supplementary_modify/dataset1/train_GT/SEG/'
test_x_path = './supplementary_modify/dataset1/test/'
train_x = []
train_y = []
test_x = []
filename_train_x = os.listdir(train_x_path)
train_num = len(filename_train_x)
for i in tqdm(range(train_num)):
    filename = filename_train_x[i]
    img = cv2.imread(train_x_path + filename, -1)
    img = img_standardization(img)
    train_x.append(img)
    filename2 = 'man_seg' + filename[1:]
    img_label = cv2.imread(train_y_path + filename2, -1)
    img_mask = generate_mask(img_label)
    train_y.append(img_mask)
filename_test_x = os.listdir(test_x_path)
filename_test_x.sort()
test_num = len(filename_test_x)
for i in tqdm(range(test_num)):
    filename = filename_test_x[i]
    img_test = cv2.imread(test_x_path + filename, -1)
    img_test = img_standardization(img_test)
    test_x.append(img_test)
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)

100%|██████████| 33/33 [00:00<00:00, 50.53it/s]


 ## 数据处理与训练

In [6]:
def train_one_fold(model, train_loader, val_loader=None, in_channel=3 , out_channel=1, fold=0):
    # 超参数设置
    lr = 0.0001
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model.apply(init_weights)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(),lr=lr)
    loss = BCEDiceLoss()
    # js停止上升时改变学习率
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
    num_epochs = 15
    for epoch in range(num_epochs):
        train_loader = tqdm(train_loader)
        train_loss = 0
        model.train()
        for i, (X, Y) in enumerate(train_loader):  # 使用枚举函数遍历train_loader
            X = Variable(X).to(device) #转化数据类型
            X = X.float()
            Y = Variable(Y).to(device)
            Y = Y.float()
            outs = model(X)  # 正向传播
            lossvalue = 0
            lossvalue += loss(outs, Y[:,0:out_channel,:,:])
            optimizer.zero_grad()  # 优化器梯度归零
            lossvalue.backward()  # 反向转播，刷新梯度值
            # 梯度裁剪
            nn.utils.clip_grad_value_(model.parameters(), 1)
            optimizer.step()  
            # 计算损失
            train_loss += float(lossvalue)
        val_loss = 0  # 定义验证损失
        val_js = 0
        max_js = 0
        model.eval() #模型转化为评估模式
        if not val_loader: continue
        val_loader = tqdm(val_loader)
        for i,(X, Y) in enumerate(val_loader):
            X = Variable(X).to(device)
            X = X.float()
            Y = Variable(Y).to(device)
            with torch.no_grad():
                outs = model(X)
            lossvalue = loss(outs, Y[:,0:out_channel,:,:])
            score = cal_jaccard_binary(outs, Y[:,0:out_channel,:,:])
            val_loss += float(lossvalue)
            val_js += score
        scheduler.step(val_js)
        max_js = max(max_js,val_js)
        model_name = 'model_unet_' + str(fold) + '.pkl'
        if max_js == val_js:
            torch.save(model, model_name)      
        print("epoch:" + ' ' + str(epoch))
        lr = optimizer.state_dict()['param_groups'][0]['lr']
        print("lr:", lr)
        print("train lose:" + ' ' + str(train_loss / len(train_loader)))
        print("val lose:" + ' ' + str(val_loss / len(val_loader)))
        print("val js:" + ' ' + str(val_js / len(val_loader)))
    return model
    

In [7]:
def validation(model, val_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    val_marker,val_border = [],[]
    val_y,val_x = [],[]
    for i, (X,Y) in enumerate(val_loader):
        X = Variable(X).to(device)
        X = X.float()
        Y = Variable(Y).to(device)
        with torch.no_grad():
          out = model(X)
          out_prob1 = torch.sigmoid(out[:,0,:,:])
          out_prob2 = torch.sigmoid(out[:,1,:,:])
        y_marker, y_border = out_prob1.cpu().clone().numpy()[0,:,:], out_prob2.cpu().clone().numpy()[0,:,:]
        val_marker.append(y_marker)
        val_border.append(y_border)
        x_real = X.cpu().clone().numpy()[0,0,:,:]
        y_real = np.zeros((640,640,3))
        y_real[:,:,0] = Y.cpu().clone().numpy()[0,0,:,:]
        y_real[:,:,1] = Y.cpu().clone().numpy()[0,1,:,:]
        y_real[:,:,2] = Y.cpu().clone().numpy()[0,2,:,:]        
        val_y.append(y_real)
        val_x.append(x_real)
    val_marker = np.asarray(val_marker)
    val_marker = np.expand_dims(val_marker,3)
    val_border = np.asarray(val_border)
    val_border = np.expand_dims(val_border,3)
    val_x = np.asarray(val_x)
    val_x = np.expand_dims(val_x,3)
    val_x = (val_x*255).astype(np.uint8)
    val_marker = (val_marker*255).astype(np.uint8)
    val_border = (val_border*255).astype(np.uint8)
    # 返回第二级学习器的输入(marker,,border,x)
    val_x_new = np.concatenate((val_marker ,val_border, val_x), axis=3)
    val_y = np.asarray(val_y)
    val_y = (val_y*255).astype(np.uint8)
    return val_y,val_x_new

In [8]:
def test(model, test_loader):  
    tta_model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
    test_x = []
    test_marker = []
    test_border = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i, X in enumerate(test_loader):
        X = Variable(X).to(device)
        X = X.float()
        with torch.no_grad():
          out = model(X)
          out_prob1 = torch.sigmoid(out[:,0,:,:])
          out_prob2 = torch.sigmoid(out[:,1,:,:])
        y_marker, y_border = out_prob1.cpu().clone().numpy()[0,:,:], out_prob2.cpu().clone().numpy()[0,:,:]
        x_real = X.cpu().clone().numpy()[0,0,:,:]
        test_marker.append(y_marker)
        test_border.append(y_border)
        test_x.append(x_real)
    test_marker = np.asarray(test_marker)
    test_border = np.asarray(test_border)
    test_x = np.asarray(test_x)
    test_marker = np.expand_dims(test_marker,3)
    test_border = np.expand_dims(test_border,3)
    test_x = np.expand_dims(test_x,3)
    test_x = (test_x*255).astype(np.uint8)
    test_marker = (test_marker*255).astype(np.uint8)
    test_border = (test_border*255).astype(np.uint8)
    test_x_new = np.concatenate((test_marker, test_border, test_x), axis=3)
    return test_x_new

## 第一级网络训练

In [9]:
n_fold = 5
# 5折交叉验证
folds=StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1)    
BATCH_SIZE = 1
test_dataset = MyDataSet(test_x, type='test')
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# 存储第一级学习器的输出结果
all_test_res = np.zeros((n_fold,test_x.shape[0],640,640,3))
all_train_res = np.zeros((train_x.shape[0],640,640,3))
all_train_real = np.zeros((train_y.shape[0],640,640,3))
train_num = train_x.shape[0]
split_x = np.ones((train_num, 10))
split_y = np.ones((train_num, 1))
fold = 0
for train_idx, val_idx in folds.split(split_x, split_y):
    train_x_data, val_x_data = train_x[train_idx], train_x[val_idx]
    train_y_data, val_y_data = train_y[train_idx], train_y[val_idx]   
    train_dataset = MyDataSet(train_x_data, train_y_data)
    val_dataset = MyDataSet(val_x_data, val_y_data)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    # 初始化模型
    out_channel = 2
    in_channel = 1
    deep_supervision=False
    # model = UNetPlusPlus(in_channel, out_channel, deep_supervision)
    model = U_Net(in_channel,out_channel, 'UNet')
    model = train_one_fold(model, train_loader, val_loader, in_channel, out_channel, fold)
    model_name = 'model_unet_' + str(fold) + '.pkl'
    model = torch.load(model_name)        
    # 输出每一折验证集和测试集的结果
    # 与最开始的输入拼接输入下一级模型
    val_real,val_res = validation(model, val_loader)
    test_res = test(model, test_loader)
    all_train_res[val_idx,:,:,:] = val_res
    all_train_real[val_idx,:,:,:] = val_real
    all_test_res[fold] = test_res
    fold += 1
    # plt.figure()
    # plt.imshow(all_train_res[val_idx[0],:,:,0])
    # plt.figure()
    # plt.imshow(all_train_res[val_idx[0],:,:,1])
    # plt.figure()
    # plt.imshow(all_train_real[val_idx[0],:,:,0])

## 第二级网络训练

In [10]:
# stacking的第二级模型
new_test_res = np.mean(all_test_res, axis=0)
new_test_res = new_test_res.astype(np.uint8)
new_train_res = all_train_res.astype(np.uint8)
new_train_real = all_train_real.astype(np.uint8)
# print(new_test_res.shape)
# print(new_test_res.dtype)

In [11]:
second_train_x = new_train_res
second_train_y = new_train_real
second_test_x = new_test_res
second_train_x2 = second_train_x[:,6:634,6:634,:]
second_train_y2 = second_train_y[:,6:634,6:634,:]
second_test_x2 = second_test_x[:,6:634,6:634,:]
second_train = MyDataSet(second_train_x2, second_train_y2)
print(second_train_x2.shape)
print(second_train_y2.shape)
second_train_dataset, second_val_dataset = random_split(second_train, [140, 35])
second_train_loader = DataLoader(second_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
second_val_loader = DataLoader(second_val_dataset, batch_size=BATCH_SIZE, shuffle=False)
second_test_dataset = MyDataSet(second_test_x2, type='test')
second_test_loader = DataLoader(second_test_dataset, batch_size=BATCH_SIZE, shuffle=False)   
# print(train_loader.dataset[0][1].shape)
print(second_train_loader.dataset[0][1].shape)
print(second_test_dataset[0].shape)

(175, 628, 628, 3)
(175, 628, 628, 3)
torch.Size([3, 640, 640])
torch.Size([3, 640, 640])


In [12]:
# # 训练第二级的U-net
model2 = U_Net(3,2, 'UNet')
# model2 = UNetPlusPlus(3, 2, False)
model2 = train_one_fold(model2, second_train_loader, second_val_loader, in_channel=2, out_channel=2, fold=6)

model_name = 'model_unet_6.pkl'
model2 = torch.load(model_name)
val_real, val_out = validation(model2, second_val_loader)
test_out = test(model2, second_test_loader)
print(test_out.shape)

  0%|          | 0/140 [00:00<?, ?it/s]

epoch: 0
lr: 0.0001
train lose: 0.2970646969974041
val lose: 0.2726823674780982
val js: 0.7521620061305155


  0%|          | 0/140 [00:00<?, ?it/s]

epoch: 1
lr: 0.0001
train lose: 0.22582335365670067
val lose: 0.24218281166894096
val js: 0.7509808395745072


  0%|          | 0/140 [00:00<?, ?it/s]

...

epoch: 12
lr: 0.0001
train lose: 0.13176092790173632
val lose: 0.165105282834598
val js: 0.76354379632912


  0%|          | 0/140 [00:00<?, ?it/s]

epoch: 13
lr: 1e-05
train lose: 0.1307175007781812
val lose: 0.16344931913273675
val js: 0.7654474075839768


100%|██████████| 35/35 [00:14<00:00,  2.47it/s]


epoch: 14
lr: 1e-05
train lose: 0.12552042143153294
val lose: 0.15881217143365314
val js: 0.7719109611496124
(33, 640, 640, 3)
